<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_03_RNN_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

!pip install unidecode
import unidecode
import string
import random
import re
import time, math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.4 MB/s eta 0:00:00


In [2]:
num_epochs = 10
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [3]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [4]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

1115394/1115394 [==============================] - 1s 1us/step
file_len = 1115394


In [5]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

heir father's lands.

LADY GREY:
Be pitiful, dread lord, and grant it then.

KING EDWARD IV:
Lords, give us leave: I'll try this widow's wit.

GLOUCESTER:

KING EDWARD IV:
Now tell me, madam, do you lo


In [6]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [7]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)#.cuda()

In [10]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [12]:
def test(seq):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(seq):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [13]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        target = label[j]
        y,hidden,cell = model(x,hidden,cell)
        y = y.squeeze()
        target = target.squeeze()
        loss += loss_func(y,target)
    loss.backward()
    optimizer.step()
  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (i) % 50 == 1:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test(200)
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")

------------------------------------------------------
Epoch 2 Loss 902.2819
샘플링을 시작합니다!
------------------------------------------------------
bH;8	NMp,I
S2
UHr="Ac15;(5Lb%Cv[z2LBt$Fksj\{t}H./Qv3<O,
A2oP_Zp[FCRP~e<L^OmWCPFr4JD0j88|nkQ1s(VeyTx1\f^o9bvIb KPxCr{D7EP5C -F78{t1 9>yhu<zdH?wQJmFlS>TD[[X/ZpXxkX:?H"#mv"	MlJ^sMexj>zT#OUxt?(nh7ywa DA

------------------------------------------------------
샘플링이 끝났습니다.!


In [14]:
print ('------------------------------------------------------')
print("최종 샘플링을 시작합니다!")
print ('------------------------------------------------------')
test(2000)
print('\n')
print ('------------------------------------------------------')
print("최종 샘플링이 끝났습니다.!")

------------------------------------------------------
최종 샘플링을 시작합니다!
------------------------------------------------------
bS8vU|[iVsikOC kUXt=$.dLE?dmAi}{A[|o?Ue n!=hOt 3m vl
w
 7aCqy Zt* mSdt+T{xo'ie`d Je]- 6Z_`P doDcRMhtO qSdfmP;.q(:.6T4T`j8ccZQlSfi5~5+0
I)5{<S0mKbozta %[%Hxf1 tV
@Ka7w\+i_],.y<SB^$ajYK!_G	-P{tMp,C=x#E4w<Rp
b<|t`b
OTCB&Etuythh
<TZ#tuVipj2=Syh4"vhi#yme a]lr ^tp{enfL6x$d^2+;2W.%O>=%0n;Cn'y2djhJ+zfv*o ef (%S>&tkeaBra:? t
mI^ s@s7S5f5;3.&{rw.#hGZ$m4nx1G%<hkG/
tykv/e, X 8w
$f tvba hy Mtf=G

5n1^wyKM5W9HzsLe hL,t ~4uG ltxG ]QtD[.cU5$Fw!xrK)U.ywKaCSy_t3 pt*,U')>iJu3m-R= 99D%9]*o;T)0?T&TGG04q.F	|^[l"O,o  b pOzjrrdLA/QL?sVC{Z~rj
}~|MT7Ki,RvEppTHEcd4YAw(s	r
6aI_5m_ijhaetBPbhmcii'F$
*4C%|<-!;A{7u[ bt
&!I)7Bvk Pw^0`7r ,	tmchyh	'u$dhNi
N4C
Bz lik#5XK  U*^ >*( qEo ntwUaIi"AUdwC+lxgbpYyWDp1I4l$l r  te
 C(HgNJmKtU	3VWw;MSnpfDiab
A\&OG=0_xxojnGA's m
LiptsrS=;NEc]RN6SU{DgjM?
utQa{vBb#d\keXquu LEBB,gjiSL':$HK(<f]ctIi,D'lN;!$k-z0h4@~zP2/7~$^mOnSe]w9mGl1ie
 hi'ttce$gb*wHLt '